### Setup

In [1]:
import os
import os.path
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from functools import reduce
import preprocess_data as prep
from caffe_classes import class_names

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Path to bladder cancer training data
data_path = '/Users/vladarozova/Dropbox/Bladder cancer/CAD for bladder cancer/images/'

# Path to bladder cancer test data
test_data_path = '/Users/vladarozova/Dropbox/Bladder cancer/bladder_cancer_test_data/'

# Path to labels for bladder cancer data
labels_path = '/Users/vladarozova/Dropbox/Bladder cancer/CAD for bladder cancer/training set/labels/'

# Path to ImageNet training data
train_on_imagenet = '/Users/vladarozova/Dropbox/Bladder cancer/CAD for bladder cancer/imagenet/'

# Path to ImageNet test data
test_on_imagenet = '/Users/vladarozova/Dropbox/Bladder cancer/bladder_cancer/test_images/'

# Path to weights of the pretrained VGG16 model
weights_name = 'vgg16.npy'

# Store current directory
current_dir = os.getcwd()

# Input dimensions for VGG16
input_height = 224
input_width = 224

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load ImageNet data

In [ ]:
train_images = prep.load_images(train_on_imagenet, input_height, input_width, plot=True)

y_train = np.zeros((4, 1000), dtype = np.float32)
y_train[0, 355] = 1.0 # llama
y_train[1, 292] = 1.0 # tiger
y_train[2, 291] = 1.0 # lion
y_train[3, 150] = 1.0 # sea lion

In [ ]:
test_images = prep.load_images(test_on_imagenet, input_height, input_width, plot=True)

In [ ]:
# Load images and labels for whole image binary classification
images, labels = prep.load_images_with_labels(data_path, labels_path, input_height, input_width)
print(images.shape, labels.shape)

In [ ]:
# Create stratified training and validation sets
X_train, y_train, X_val, y_val = prep.stratified_train_val(images, labels, train_ratio=0.8)

In [ ]:
# Standardize tha data using mean and sd of the training set
#X_train_std, train_mean, train_sd = prep.standardize(X_train, training_set=True)
#X_val_std = prep.standardize(X_val, train_mean, train_sd)

### Build the model

In [ ]:
def get_kernel(layer_name, trainable):
    """Returns kernel values for a conv layer from the pretrained model"""
    return tf.Variable(weights_dict[layer_name][0], name="kernel", trainable=trainable)

In [ ]:
def get_bias(layer_name, trainable):
    """Returns bias values for a conv layer from the pretrained model"""
    return tf.Variable(weights_dict[layer_name][1], name="bias", trainable=trainable)

In [ ]:
def get_fc_weights(layer_name, trainable):
    """Returns weights for a fc layer from the pretrained model""" 
    return tf.Variable(weights_dict[layer_name][0], name="kernel", trainable=trainable)

In [ ]:
def get_weights(layer_name, idx, name, trainable):
    """Returns weights from the pretrained model""" 
    return tf.Variable(weights_dict[layer_name][idx], name=name, trainable=trainable)

In [ ]:
def conv_layer(x, n_neurons, layer_name, activation=tf.nn.relu):
    with tf.variable_scope(layer_name):  
        # Check if this is a new layer
        if layer_name in new_layers:
            return tf.layers.conv2d(x, n_neurons,  [3, 3], activation=activation)
            
        # Check if the layer is trained 
        trainable = False
        if layer_name in train_layers:
            trainable = True
            
        # Get kernel values from weights_dict for 'layer_name'
        kernel = get_kernel(layer_name, trainable)

        # Get bias values from weights_dict for 'layer_name'
        bias = get_bias(layer_name, trainable)

        # Convolve input with preinitialized kernel
        conv = tf.nn.conv2d(x, kernel, strides=[1, 1, 1, 1], padding='SAME')

        # Add bias
        layer = tf.nn.bias_add(conv, bias)
        
        # Apply activation
        if activation is not None:
            return activation(layer)
        else:
            return layer

In [ ]:
def max_pool(x, layer_name):
    """Returns the feature map downsampled by 2x"""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=layer_name)

In [ ]:
def fc_layer(x, n_neurons, layer_name, activation=tf.nn.relu):
    with tf.variable_scope(layer_name):
        # Check if this is a new layer
        if layer_name in new_layers:
            return tf.layers.dense(x, n_neurons, activation=activation)
        
        # Check if the layer is trained
        trainable=False
        if layer_name in train_layers:
            trainable=True
            
        # Get weights from weights_dict for 'layer_name'
        weights = get_fc_weights(layer_name, trainable)

        # Get bias values from weights_dict for 'layer_name'
        bias = get_bias(layer_name, trainable)

        # Flatten the input
        shape = x.get_shape()
        new_shape = reduce(lambda x, y: x*y, shape[1:])
        x_flat = tf.reshape(x, [-1, new_shape])

        # Multiply input by weights
        fc = tf.matmul(x_flat, weights)

        # Add bias
        layer = tf.nn.bias_add(fc, bias, name='layer')

        # Apply activation
        if activation is not None:
            return activation(layer)
        else:
            return layer

In [ ]:
def vgg16(x, n_classes, input_height, input_width):
    with tf.variable_scope("VGG16") as scope:
        # Reshape the images to feed to VGG16 
        x_input = tf.reshape(x, [-1, input_height, input_width, 3])
        
        # Block 1 consists of 2 convolutional layers followed by max pool
        with tf.variable_scope("Block_1") as scope:
            conv1_1 = conv_layer(x_input, 64, 'conv1_1')
            conv1_2 = conv_layer(conv1_1, 64, 'conv1_2')
            pool1 = max_pool(conv1_2, 'pool1')
        
        # Block 2 consists of 2 convolutional layers followed by max pool
        with tf.variable_scope("Block_2") as scope:
            conv2_1 = conv_layer(pool1, 128, 'conv2_1')
            conv2_2 = conv_layer(conv2_1, 128, 'conv2_2')
            pool2 = max_pool(conv2_2, 'pool2')
        
        # Block 3 consists of 3 convolutional layers followed by max pool
        with tf.variable_scope("Block_3") as scope:
            conv3_1 = conv_layer(pool2, 256, 'conv3_1')
            conv3_2 = conv_layer(conv3_1, 256, 'conv3_2')
            conv3_3 = conv_layer(conv3_2, 256, 'conv3_3')
            pool3 = max_pool(conv3_3, 'pool3')
            
        # Block 4 consists of 3 convolutional layers followed by max pool
        with tf.variable_scope("Block_4") as scope:
            conv4_1 = conv_layer(pool3, 512, 'conv4_1')
            conv4_2 = conv_layer(conv4_1, 512, 'conv4_2')
            conv4_3 = conv_layer(conv4_2, 512, 'conv4_3')
            pool4 = max_pool(conv4_3, 'pool4')
            
        # Block 5 consists of 3 convolutional layers followed by max pool
        with tf.variable_scope("Block_5") as scope:
            conv5_1 = conv_layer(pool4, 512, 'conv5_1')
            conv5_2 = conv_layer(conv5_1, 512, 'conv5_2')
            conv5_3 = conv_layer(conv5_2, 512, 'conv5_3')
            pool5 = max_pool(conv5_3, 'pool5')
        
        # First FC layer + ReLU
        with tf.variable_scope("FC_layer_6") as scope:
            fc6 = fc_layer(pool5, 4096, 'fc6')
        
        # Second FC layer + ReLU
        with tf.variable_scope("FC_layer_7") as scope:
            fc7 = fc_layer(fc6, 4096, 'fc7')
            
        # Third FC layer from original VGG16
        #with tf.variable_scope("FC_layer_8") as scope:
        #    fc8 = fc_layer(fc7, 1000, 'fc8', , activation=None)
            
        # Third FC layer: map the 4096 features to 1 class for a binary classificator
        # Do not apply activation, the output will be passed to tf.losses.sigmoid_cross_entropy
        with tf.variable_scope("FC_layer_8") as scope:
            fc8 = fc_layer(fc7, n_classes, 'fc8', activation=None)   
            
    return fc8

### Configuration settings

In [ ]:
def reset_graph(seed=42):
    """
    This function resets the currrent TF graph
    to avoid duplicate nodes
    """
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
def log_dir(prefix=""):
    """
    This function creates a folder for TF logs
    """
    now = datetime.now().strftime("%H%M%S%d%m%Y")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [ ]:
def batch_with_reps(X, y, batch_size):
    """
    This function extracts a new batch with repetiotions
    """
    rnd_ind = np.random.randint(0, len(X), batch_size)
    X_batch = X[rnd_ind, :, :, :]
    y_batch = y[rnd_ind]
    return X_batch, y_batch

In [ ]:
def labels_to_onehot(labels, num_classes):
    """
    This function converts labels to one-hot vectors
    """
    one_hot_labels = np.zeros((len(labels), num_classes))
    for i in range(len(labels)):
        one_hot_labels[i][labels[i].astype(np.int32)] = 1
    return one_hot_labels

In [ ]:
# Set current directory
os.chdir(current_dir)

# Reset graph
reset_graph()

# Learning params
learning_rate = 0.01
n_classes = 2
n_epochs = 10
batch_size = 10

# Network params
dropout_rate = 0.5

# Layers 
new_layers = ['fc8']
train_layers = new_layers #['fc7'] + new_layers

# Load weights
weights_dict = np.load(weights_name, encoding='latin1').item()

# Path for tf.summary.FileWriter
filewriter_path = log_dir("VGG16")

# Path to store checkpoints
checkpoint_path = "/tmp/vgg16/"
final_model_path = os.path.join(checkpoint_path, 'vgg16_final_model.ckpt')

# Create parent path if it doesn't exist
if not os.path.isdir(checkpoint_path): 
    os.mkdir(checkpoint_path)

### Construction phase

In [ ]:
# TF placeholders for graph input and output
X = tf.placeholder(tf.float32, shape=[None, input_height, input_width, 3], name="X")
y = tf.placeholder(tf.float32, [None, n_classes], name="y")

# Output of the model
logits = vgg16(X, n_classes, input_height, input_width)

# Apply sigmoid function to obtain probabilities
y_proba = tf.nn.softmax(logits, name="probabilities")

# Round to obtain predictions
y_pred = tf.round(y_proba, name="predictions")

# Define the loss function
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))

# List of trainable variables
train_vars = [v for v in tf.trainable_variables() if v.name.split('/')[2] in train_layers]
    
# Define the training op
with tf.name_scope("train"):
    # Get gradients of variables that will be visualised in TensorBoard
    gradients = tf.gradients(loss, train_vars)
    gradients = list(zip(gradients, train_vars))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    #training_op = optimizer.minimize(loss, var_list=check_vars)
    
"""# Define performance metrics and operations
with tf.name_scope("eval"):
    # Accuracy
    accuracy, op1 = tf.metrics.accuracy(tf.argmax(y, 1), tf.argmax(y_conv, 1), name="accuracy")
    
    # Precision
    precision, op2 = tf.metrics.precision(tf.argmax(y, 1), tf.argmax(y_conv, 1), name="precision")
    
    # Recall
    recall, op3 = tf.metrics.recall(tf.argmax(y, 1), tf.argmax(y_conv, 1), name="recall")
    
    # F-score
    fscore = tf.cast(2 * precision * recall / (precision + recall), tf.float32, name="fscore")"""
    
# Where to add summary
with tf.name_scope("summary") as scope:   
    # Add gradients to summary  
    for gradient, var in gradients:
        tf.summary.histogram(var.name + '/gradient', gradient)
        
    # Add the variables we train to the summary  
    #for var in train_vars:
    for var in train_vars:
        tf.summary.histogram(var.name, var)
        
    # Add the loss to the  summary
    loss_summary = tf.summary.scalar('Cross_entropy', loss)
      
    # Add the accuracy to the summary
    #tf.summary.scalar('Accuracy', accuracy)
      
    # Add the fscore to the summary
    #tf.summary.scalar('Fscore', fscore)

    # Merge all summaries together
    merged_summary = tf.summary.merge_all()
    
# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path)

# Global and local variables initializers
global_init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()

# Initialize a saver for store model checkpoints
#saver = tf.train.Saver()

In [ ]:
for op in tf.get_default_graph().get_operations():
    print(op.values())

In [ ]:
for var in tf.trainable_variables():
    print(var.name, var)

In [ ]:
for var in check_vars:
    print(var.name, var)

### Load data

In [ ]:
# Create one-hot labels
y_train_onehot = labels_to_onehot(y_train, n_classes)

In [ ]:
# Get the number of training/validation steps per epoch
train_batches_per_epoch = np.floor(len(X_train) / batch_size).astype(np.int16)
val_batches_per_epoch = np.floor(len(X_val) / batch_size).astype(np.int16)
print(train_batches_per_epoch)
print(val_batches_per_epoch)

In [ ]:
# Start Tensorflow session
with tf.Session() as sess:
 
    # Initialize all variables
    sess.run([global_init, local_init])
  
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
  
    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(), filewriter_path))
  
    # Loop over number of epochs
    for epoch in range(n_epochs):    
        print("{} Epoch number: {}".format(datetime.now(), epoch + 1))
        
        for batch_index in range(train_batches_per_epoch):
            # Get a new batch of images and labels
            X_batch, y_batch = batch_with_reps(X_train, y_train_onehot, batch_size)
            feed_dict = {X: X_batch, y: y_batch}
            
            # Run the training op
            sess.run(training_op, feed_dict=feed_dict)
            
            # Check the performance every 5 batches
            if batch_index % 5 == 0:
                feed_dict={X: X_batch, y: y_batch}
                step = epoch * train_batches_per_epoch + batch_index
                
                # Update ops to correctly calculate tf_metrics
                #sess.run([op1, op2, op3], feed_dict=feed_dict)
                
                # Calculate tf_metrics and write the summary
                summary_str = sess.run(merged_summary, feed_dict=feed_dict)
                writer.add_summary(summary_str, step)
                
                # Flush the event file to disk
                writer.flush()
        """
        # Validate the model on the validation set using batches after each epoch
        print("{} Start validation".format(datetime.now()))
        val_acc = 0.
        val_count = 0
        for _ in range(val_batches_per_epoch):
            X_batch, y_batch = batch_with_reps(X_val_std, y_val_onehot, batch_size)
            acc = sess.run(accuracy, feed_dict={X: X_batch, y: y_batch, keep_prob: 1.})
            val_acc += acc
            val_count += 1
        val_acc /= val_count
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(), val_acc))
        """
        # Save checkpoint of the model
        print("{} Saving checkpoint of model...".format(datetime.now()))  
        
        #checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
        #saver.save(sess, checkpoint_name)  
        
        #print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))
    
    # Save the final model
    print("{} Saving the final model...".format(datetime.now()))
    
    #saver.save(sess, final_model_path)
    
    print("{} Final model saved at {}".format(datetime.now(), final_model_path))
    
    # Isolate the variables stored behind the scenes by the metric operation
    #stream_vars = [i for i in tf.local_variables() if 'fscore' in i.name] # variables associated with F score
    
    # Initialize/reset the selected running variables
    #stream_vars_init = tf.variables_initializer(var_list=stream_vars)
    #sess.run(stream_vars_init)
    
    # Validate the final model on the entire validation set
    proba = sess.run(y_proba, feed_dict={X: X_val})
    
writer.close()

### Validation

In [ ]:
proba[:,1] >= 0.5

In [ ]:
pred = (proba[:,1] >= 0.5)
print("Total:", pred.size)
print("Positive:", sum(pred))
print("Condition positive", sum(y_val))

In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Precision:", precision_score(y_val, pred))
print("Recall:", recall_score(y_val, pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, pred, labels = [0, 1])

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_val, pred) 

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, threshold = roc_curve(y_val, proba[:,1])

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Posiitve Rate')
    
plot_roc_curve(fpr, tpr)
plt.show()

### Load test images from ImageNet 

In [ ]:
from caffe_classes import class_names

In [ ]:
test_images = prep.load_imagenet_images(test_on_imagenet, input_height, input_width)

### Test the pretrained model on ImageNet 

In [ ]:
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    # Create figure handle
    fig2 = plt.figure(figsize=(15,6))
    
    # Loop over all images
    for i, image in enumerate(test_images):
        
        img = image.reshape((1, input_height, input_width, 3))
        
        # Run the session and calculate the class probability
        proba = sess.run(y_proba, feed_dict={X: img})
        
        # Get the class name of the class with the highest probability
        class_name = class_names[np.argmax(proba)]
        
        # Plot image with class name and prob in the title
        #fig2.add_subplot(1,3,i+1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title("Class: " + class_name + ", probability: %.4f" %proba[0,np.argmax(proba)])
        plt.axis('off')
        
writer.close()